In [1]:
import warnings
warnings.filterwarnings("ignore")

### Data Preparation

In [2]:
from datasets import load_dataset, get_dataset_split_names

# Load an example dataset: 
# dataset = load_dataset('librispeech_asr', 'clean', split='test', cache_dir='I:\Repos\HFdatasets', ignore_verifications=True)
dataset = load_dataset('timit_asr', data_dir='I:\\Repos\\HFdatasets\\downloads_manual\\timit', cache_dir='I:\Repos\HFdatasets')

# Remove redundant columns
dataset = dataset.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

# Only use a subset
# dataset['train'] = dataset['train'].filter(lambda x, idx: idx < 1000, with_indices=True)
# dataset['test'] = dataset['test'].filter(lambda x, idx: idx < 400, with_indices=True)

# Print dataset information
dataset['train'][0]

Using custom data configuration default-8cee652d49042a5b
Found cached dataset timit_asr (I:/Repos/HFdatasets/timit_asr/default-8cee652d49042a5b/0.0.0/43f9448dd5db58e95ee48a277f466481b151f112ea53e27f8173784da9254fb2)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text'],
        num_rows: 1680
    })
})

In [3]:
# Clean data of special characters and normalise to lowercase
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def clean_data(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

dataset = dataset.map(clean_data)

Loading cached processed dataset at I:/Repos/HFdatasets/timit_asr/default-8cee652d49042a5b/0.0.0/43f9448dd5db58e95ee48a277f466481b151f112ea53e27f8173784da9254fb2\cache-96542c3dd8ab44e4.arrow
Loading cached processed dataset at I:/Repos/HFdatasets/timit_asr/default-8cee652d49042a5b/0.0.0/43f9448dd5db58e95ee48a277f466481b151f112ea53e27f8173784da9254fb2\cache-7b9b6096fbd69b4f.arrow


In [4]:
# Display random data elements for demonstration
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(dataset['train'].remove_columns(["file", "audio"]))

,text
0,it helps those people who help themselves
1,by eating yogurt you may live longer
2,a huge power outage rarely occurs
3,tragedy presumes such a configuration
4,trish saw hours and hours of movies saturday
5,don't ask me to carry an oily rag like that
6,would a tomboy often play outdoors
7,a young mouse scampered across the field and disappeared
8,just drop notices in any suggestion box
9,angora cats are furrier than siamese


In [5]:
# Play an example audio file
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(dataset["train"]))

print(dataset["train"][rand_int]["text"])
ipd.Audio(data=np.asarray(dataset["train"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

oh you'll wobble and weave quite a bit at first


### Model Initialisation

A Wav2Vec2 model contains the model, a trained tokenizer, and a feature extractor. 

The feature extractor is basically just a list of algorithms which transform and preprocess the input waveforms. The model contains the pre-trained transformer with attention heads.

The tokenizer is what we are training. This is a neural network layer which will generate a sequence of tokens based on the audio data.

In [6]:
# First we need to define our vocabulary. For normal spoken speech, the common approach is to define a letter- or phoneme-level tokenizer.
# Here, we will train a letter-level tokenizer.
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

# Set up vocabulary
vocabs = dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=dataset.column_names["train"])
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}

# Add blank tokens
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

# Save to file
import json
with open('timit_vocab.json', 'w') as vocab_file:
  json.dump(vocab_dict, vocab_file)

print(len(vocab_dict))
vocab_dict

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

30


{'r': 0,
 'n': 1,
 'u': 2,
 'p': 3,
 'j': 4,
 'g': 6,
 'a': 7,
 's': 8,
 'f': 9,
 'y': 10,
 'b': 11,
 't': 12,
 'o': 13,
 'k': 14,
 'v': 15,
 'l': 16,
 "'": 17,
 'd': 18,
 'i': 19,
 'q': 20,
 'm': 21,
 'e': 22,
 'w': 23,
 'c': 24,
 'h': 25,
 'x': 26,
 'z': 27,
 '|': 5,
 '[UNK]': 28,
 '[PAD]': 29}

In [7]:
model_checkpoint = "facebook/wav2vec2-base"

In [8]:
# Secondly, we will create the tokenizer object. This is the thing that we are fine-tuning.
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_checkpoint)

tokenizer_type = config.model_type if config.tokenizer_class is None else None
config = config if config.tokenizer_class is not None else None

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
  "./",
  config=config,
  tokenizer_type=tokenizer_type,
  unk_token="[UNK]",
  pad_token="[PAD]",
  word_delimiter_token="|",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
# Thirdly, we create the feature extractor. This brings the fine-tuning data into the same format as the pre-training data.
# As such, the parameters of the feature extractor depends entirely on the specs of the pre-trained model. 
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)

# from transformers import Wav2Vec2FeatureExtractor

# feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [10]:
# Define a processor object. This is just a wrapper class that provides a pipeline for inference. 
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [11]:
# Prepare input and output
def prepare_dataset(batch):
    audio = batch["audio"]

    # Load and resample input data
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    # batch["input_length"] = len(batch["input_values"])
    # Load text transcription as label
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

dataset = dataset.map(prepare_dataset, remove_columns=dataset.column_names["train"])
dataset

  0%|          | 0/4620 [00:00<?, ?ex/s]

  0%|          | 0/1680 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['input_values', 'labels'],
        num_rows: 1680
    })
})

In [12]:
# # Filter out training samples longer than 4 seconds, to save memory
# max_input_length_in_sec = 4.0
# dataset["train"] = dataset["train"].filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

In [13]:
# Prepare a data collator, which will apply separate padding modalities to input and output
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union


class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """
    def __init__(self, 
        processor: Wav2Vec2Processor,
        padding: Union[bool, str] = True,
        max_length: Optional[int] = None,
        max_length_labels: Optional[int] = None,
        pad_to_multiple_of: Optional[int] = None,
        pad_to_multiple_of_labels: Optional[int] = None,
    ) -> None:
        self.processor = processor
        self.padding = padding
        self.max_length = max_length
        self.max_length_labels = max_length_labels
        self.pad_to_multiple_of = pad_to_multiple_of
        self.pad_to_multiple_of_labels = pad_to_multiple_of_labels

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

### Model Training

In [14]:
# Define an evaluation metric. In this case we use a logit vector of the same length as our vocabulary. 

from datasets import load_metric

wer_metric = load_metric("wer")

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [15]:
# Finally we can load the pre-trained transformer model
from transformers import AutoModelForCTC

model = AutoModelForCTC.from_pretrained(
    model_checkpoint, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)
model.freeze_feature_extractor()

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_q.weight', 'quantizer.weight_proj.bias', 'project_hid.weight', 'project_hid.bias', 'project_q.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [16]:
from transformers import TrainingArguments

repo_name = "wav2vec2-timit-test"

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True, 
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  push_to_hub=False,
)

In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


In [18]:
# 90195872 parameters
trainer.train()

***** Running training *****
  Num examples = 4620
  Num Epochs = 30
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 8670
  Number of trainable parameters = 90195872


  0%|          | 0/8670 [00:00<?, ?it/s]

***** Running Evaluation *****
  Num examples = 1680
  Batch size = 8


{'loss': 3.5104, 'learning_rate': 4.9800000000000004e-05, 'epoch': 1.73}


  0%|          | 0/210 [00:00<?, ?it/s]

...

Saving model checkpoint to wav2vec2-timit-test\checkpoint-8500
Configuration saved in wav2vec2-timit-test\checkpoint-8500\config.json


{'eval_loss': 0.4538317620754242, 'eval_wer': 0.278547308938047, 'eval_runtime': 72.6546, 'eval_samples_per_second': 23.123, 'eval_steps_per_second': 2.89, 'epoch': 29.41}


Model weights saved in wav2vec2-timit-test\checkpoint-8500\pytorch_model.bin
Feature extractor saved in wav2vec2-timit-test\checkpoint-8500\preprocessor_config.json
Deleting older checkpoint [wav2vec2-timit-test\checkpoint-7500] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 9093.9923, 'train_samples_per_second': 15.241, 'train_steps_per_second': 0.953, 'train_loss': 0.3200886911724403, 'epoch': 30.0}


TrainOutput(global_step=8670, training_loss=0.3200886911724403, metrics={'train_runtime': 9093.9923, 'train_samples_per_second': 15.241, 'train_steps_per_second': 0.953, 'train_loss': 0.3200886911724403, 'epoch': 30.0})

In [19]:
# Compute error metric
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

results = dataset["test"].map(map_to_result, remove_columns=dataset["test"].column_names)

  0%|          | 0/1680 [00:00<?, ?ex/s]

In [20]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.255
